# AI Disciplinarian
This program detects whether the child is wearing proper uniform, improper uniform or uniform with certain elements missing. This is a prototype which can be trained on data of various schools' uniforms to apply for specific use cases.

# Data Acquisition
The data was collected manually via google search engine for all the classes.

# Modelling

In [1]:
#Import the required libraries
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np
from sklearn.metrics import classification_report
import os
import cv2

In [2]:
#Import the CNN
np.set_printoptions(suppress=True)
model = tensorflow.keras.models.load_model('keras_model.h5')

In [3]:
#Image Processing
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
labels = ["Proper Uniform","Tie Missing","Improper Uniform"]

def imagePreparation(image):
    #resize the image and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image_array

# Evaluation
On evaluation, the model gives a 100% accuracy for the given test dataset.

In [4]:
# Create the Evaluation Variables
realtestY = np.array(["Improper Uniform","Improper Uniform","Improper Uniform","Improper Uniform","Proper Uniform","Proper Uniform","Proper Uniform","Tie Missing","Tie Missing","Tie Missing"])
filenames = []
predictedY = []
path = "Data/Test/"

def evaluateaccuracy(filenames,predictedY):
    predictedY = np.array(predictedY)
    if (np.sum(realtestY!=predictedY)>0):
        print ("Wrong Predictions: (filename, labelled, predicted) ")
        print (np.dstack([filenames,realtestY,predictedY]).squeeze()[(realtestY!=predictedY)])
    return "Correct :"+ str(np.sum(realtestY==predictedY)) + ". Wrong: "+str(np.sum(realtestY!=predictedY)) + ". Correctly Classified: " + str(np.sum(realtestY==predictedY)*100/len(predictedY))+"%"

for filename in os.listdir(path):
    img = Image.open(path+filename)
    imagePreparation(img)
    prediction = model.predict(data)
    result = np.argmax(prediction[0])
    
    print (filename + ": " + labels[result])
    filenames.append(filename)
    predictedY.append(labels[result])

#Print the Evaluation Report
print ()
print(classification_report(realtestY, predictedY))
print (evaluateaccuracy(filenames,predictedY))

improper 1.jpeg: Improper Uniform
improper 3.jpeg: Improper Uniform
improper 4.jpeg: Improper Uniform
improper 5.jpeg: Improper Uniform
proper 1.jpeg: Proper Uniform
proper 2.jpeg: Proper Uniform
proper 3.jpeg: Proper Uniform
tie missing 1.jpeg: Tie Missing
tie missing 2.jpeg: Tie Missing
tie missing 3.jpeg: Tie Missing

                  precision    recall  f1-score   support

Improper Uniform       1.00      1.00      1.00         4
  Proper Uniform       1.00      1.00      1.00         3
     Tie Missing       1.00      1.00      1.00         3

        accuracy                           1.00        10
       macro avg       1.00      1.00      1.00        10
    weighted avg       1.00      1.00      1.00        10

Correct :10. Wrong: 0. Correctly Classified: 100.0%


# Deployment

In [5]:
#Mode Selection
print("Choose your preferred Mode of testing:"); print("1 - Image"); print("2 - Webcam")
mode = int(input("Your Choice (1 or 2): "))

if mode == 1:
    #Getting the prediction from an image
    print("Save your image to be predicted in the same folder as this Jupyter Notebook.")
    filename = input("Enter the filename along with extension(.jpg,.png etc): ")

    testpiece = Image.open(filename)
    imagePreparation(testpiece)

    prediction = model.predict(data)
    result = np.argmax(prediction[0])
    print("Uniform Status: " + labels[result])

elif mode == 2:
    #Getting the prediction from a webcam feed
    camera = cv2.VideoCapture(0)

    while (True):
        #Getting the current frame
        ret, frame = camera.read()
        img = Image.fromarray(frame)

        #Get the Prediction
        imagePreparation(img)
        prediction = model.predict(data)
        result = np.argmax(prediction[0])

        #Show the Result
        cv2.putText(frame,"Uniform Status: " + labels[result], (0,frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)  
        cv2.imshow("Webcam Feed (Press Spacebar to Exit)", frame)

        #Exit Mechanism
        if cv2.waitKey(1) & 0xFF == ord(' '):
                break 
    camera.release()
    cv2.destroyAllWindows()
    
print(); print("Thanks for using School Uniform Checker!")

Choose your preferred Mode of testing:
1 - Image
2 - Webcam
Your Choice (1 or 2): 2

Thanks for using School Uniform Checker!


# Applications / Use Cases: 
This can be fitted in the entrance / door of the buses / vans that the school uses to provide public transport and also fit one at students' entrance gate. This can later be modelled in to a device consisting of an LED light / Buzzer, Raspberry Pie and NCS. It can buzz whenever it detects a student in improper uniform and can send a pic to teachers/authorities when a student comes in improper uniform, this can help teachers keep track of students who regularly make minor mistakes and can warn them if needed.